In [1]:
import numpy as np
import pandas as pd
import os

In [4]:
df = pd.read_csv('../mapillary_vistas_final_merged.csv')
df.shape

/home/nab_126/miniconda3/envs/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(28022, 34)

In [6]:
df = df[(df['final_shape'] != 'circle-750.0') & (df['points'].isna()) & (df['final_shape'] != 'other-0.0-0.0') & (df['final_shape'] == 'octagon-915.0')]
df[df['filename_y'] == '8lkcFc59-2RgSU203mlYEQ.jpg']

,filename,object_id,shape,predicted_shape,predicted_class,group_x,batch_number,row,column,new_class,...,ymin_ratio,tgt_final,filename_y,h0,w0,h_ratio,w_ratio,w_pad,h_pad,tgt_polygon
27968,8lkcFc59-2RgSU203mlYEQ_93.png,93,octagon,other,other-0.0-0.0,3,0,1,e,10.0,...,0.37451,"[[318, 215], [374, 207], [386, 257], [330, 265]]",8lkcFc59-2RgSU203mlYEQ.jpg,1080,1920,0.666667,0.666667,16.0,8.0,NaN


In [3]:
df = pd.read_csv('../runs/results.csv')
df = df.replace({'apply_patch': 'True', 'random_patch': 'True'}, 1)
df = df.replace({'apply_patch': 'False', 'random_patch': 'False'}, 0)
df['apply_patch'] = df['apply_patch'].astype(float).astype(bool)
df['random_patch'] = df['random_patch'].astype(float).astype(bool)
df[['fnr_octagon', 'num_octagon_labels', 'num_octagon_with_patch', 'fn_octagon', 'apply_patch', 'random_patch']]

,fnr_octagon,num_octagon_labels,num_octagon_with_patch,fn_octagon,apply_patch,random_patch
0,0.200240,834.0,571.0,167.0,True,False
1,0.201439,834.0,571.0,168.0,True,False
2,0.191596,834.0,0.0,160.0,False,False
3,0.192795,834.0,11.0,161.0,True,False
4,0.191596,834.0,11.0,160.0,True,False


In [13]:
success_rate = (164-160)/(571-0)
print('success rate with random patch', np.round(success_rate, 4))
success_rate = (172-160)/(571-0)
print('success rate with rp2 patch', np.round(success_rate, 4))


success rate with random patch 0.007
success rate with rp2 patch 0.021


In [8]:
pd.read_csv('../mapillary_vistas_final_merged.csv')

/home/nabeel/miniconda3/envs/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,filename,object_id,shape,predicted_shape,predicted_class,group_x,batch_number,row,column,new_class,...,ymin_ratio,tgt_final,filename_y,h0,w0,h_ratio,w_ratio,w_pad,h_pad,tgt_polygon
0,--NSVcUgfVhFd6uzkqHOOg_48.png,48,diamond,diamond,diamond-600.0,1,0,0,a,NaN,...,0.522794,"[[941.0980392156862, 558.3529411764705], [961....",--NSVcUgfVhFd6uzkqHOOg.jpg,2448,3264,0.392157,0.392157,16.0,16.0,NaN
1,--tczNya4_w5IhU9DeRMHw_33.png,33,diamond,diamond,diamond-915.0,1,0,0,b,NaN,...,0.362010,"[[248.94117647058803, 339.1372549019607], [305...",--tczNya4_w5IhU9DeRMHw.jpg,2448,3264,0.392157,0.392157,16.0,16.0,NaN
2,-0QdotcnWZMgDLHELjMeoQ_117.png,117,diamond,diamond,diamond-915.0,1,0,0,c,NaN,...,0.616176,"[[794.0392156862744, 695.2156862745097], [805....",-0QdotcnWZMgDLHELjMeoQ.jpg,2448,3264,0.392157,0.392157,16.0,16.0,NaN
3,-0QdotcnWZMgDLHELjMeoQ_50.png,50,diamond,diamond,diamond-915.0,1,0,0,d,NaN,...,0.594118,"[[531.6862745098036, 658.7450980392156], [546....",-0QdotcnWZMgDLHELjMeoQ.jpg,2448,3264,0.392157,0.392157,16.0,16.0,NaN
4,-2H8gOJp_LEvus2Y6pCIOA_101.png,101,diamond,diamond,diamond-915.0,1,0,0,e,NaN,...,0.363655,"[[1070.4578313253012, 310.6184738955823], [109...",-2H8gOJp_LEvus2Y6pCIOA.jpg,2988,3984,0.321285,0.321285,16.0,16.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28017,C9Cy-eUHARvbuSxTSrnhjA_155.png,155,triangle,circle,circle-750.0,2,0,0,b,NaN,...,0.432470,"[[965.0, 1543.75], [995.0, 1510.0], [1011.25, ...",C9Cy-eUHARvbuSxTSrnhjA.jpg,3936,5248,0.243902,0.243902,16.0,16.0,NaN
28018,Ka1tPXdxFv6sIJTRSst1ew_143.png,143,triangle,circle,circle-750.0,2,0,0,d,1.0,...,0.535049,"[[826, 417], [835, 430], [815, 433]]",Ka1tPXdxFv6sIJTRSst1ew.jpg,2160,3840,0.333333,0.333333,16.0,8.0,NaN
28019,cjm34RdPXo5C0UJ4IuFibg_54.png,54,triangle,circle,circle-750.0,2,0,1,d,NaN,...,0.424265,"[[577.1428571428571, 912.8571428571428], [599....",cjm34RdPXo5C0UJ4IuFibg.jpg,2448,3264,0.392157,0.392157,16.0,16.0,NaN
28020,Y2_AjNDiZPFRKv205TvqmA_36.png,36,octagon,octagon,octagon-915.0,1,0,0,a,NaN,...,0.520833,"[[883.8431372549019, 554.0392156862745], [931....",Y2_AjNDiZPFRKv205TvqmA.jpg,2448,3264,0.392157,0.392157,16.0,16.0,NaN


In [75]:
# directory = '/data/shared/mtsd_v2_fully_annotated/cropped_signs_v6/train'
# print(os.listdir(directory))
# for label in range(0, 12):
#     count = 0
#     for filename in os.listdir(os.path.join(directory, str(label))):
#         if filename.endswith(".jpg") or filename.endswith(".png"):
#             count += 1
#     print(label, count)


In [76]:
#mapillary vistas 

# model 1
# mv_df = pd.read_csv('../mapillaryvistas_model_3_no_background.csv')
# mv_df = pd.read_csv('../mapillaryvistas_model_3_no_background__.csv')
# mv_df = pd.read_csv('../mapillaryvistas_model_3.csv')
# mv_df = pd.read_csv('../mapillaryvistas_model_3_updated.csv')
mv_df = pd.read_csv('../mapillaryvistas_model_3_updated_optimized.csv')


# model 2
# mv_df = pd.read_csv('../mapillaryvistas_model_2.csv')

# mv_df = pd.read_csv('../mapillaryvistas_model_2_no_background.csv')

# only circles and triangles on rects
# mv_df = pd.read_csv('../mapillaryvistas_model_2_no_background_strict.csv')

# any shape except diamond on rects
# mv_df = pd.read_csv('../mapillaryvistas_model_2_no_background_strict_diamond.csv')

mv_df

,filename,object_id,shape,predicted_shape,predicted_class,group,batch_number,row,column
0,--NSVcUgfVhFd6uzkqHOOg_48.png,48,diamond,diamond,diamond-600.0,1,0,0,a
1,--tczNya4_w5IhU9DeRMHw_33.png,33,diamond,diamond,diamond-915.0,1,0,0,b
2,-0QdotcnWZMgDLHELjMeoQ_117.png,117,diamond,diamond,diamond-915.0,1,0,0,c
3,-0QdotcnWZMgDLHELjMeoQ_50.png,50,diamond,diamond,diamond-915.0,1,0,0,d
4,-2H8gOJp_LEvus2Y6pCIOA_101.png,101,diamond,diamond,diamond-915.0,1,0,0,e
...,...,...,...,...,...,...,...,...,...
28399,pUXCSbq7nUO1Cx4y4YZl_Q_74.png,74,triangle,octagon,octagon-915.0,2,0,2,b
28400,qBy_nd5G6qZiFDuZrNfGrA_30.png,30,triangle,diamond,diamond-915.0,2,0,2,c
28401,Y2_AjNDiZPFRKv205TvqmA_36.png,36,octagon,octagon,octagon-915.0,1,0,0,a
28402,eDLXGB6vYtAyV71JIv6JdQ_58.png,58,octagon,octagon,octagon-915.0,1,0,0,b


In [77]:
len(mv_df['filename'].unique())

28404

In [78]:
mv_df['group'].value_counts()

3    16800
1    10307
2     1297
Name: group, dtype: int64

In [79]:
mv_df['group'].value_counts()/len(mv_df)

3    0.591466
1    0.362871
2    0.045663
Name: group, dtype: float64

In [80]:
grouped_df = mv_df.groupby(by=['shape', 'predicted_shape', 'group']).count()
grouped_df = grouped_df.reset_index()
grouped_df[grouped_df['group'] == 1][['shape', 'predicted_shape', 'group', 'filename']].rename({'filename':'count'}, axis=1)

,shape,predicted_shape,group,count
0,circle,circle,1,5049
9,diamond,diamond,1,1244
18,octagon,octagon,1,3
31,pentagon,pentagon,1,84
36,rect,circle,1,903
37,rect,diamond,1,23
38,rect,octagon,1,20
40,rect,pentagon,1,3
41,rect,rect,1,1160
42,rect,square,1,657


In [81]:
grouped_df = mv_df.groupby(by=['shape', 'predicted_shape', 'group']).count()
grouped_df = grouped_df.reset_index()
grouped_df[grouped_df['group'] == 2][['shape', 'predicted_shape', 'group', 'filename']].rename({'filename':'count'}, axis=1)

,shape,predicted_shape,group,count
1,circle,diamond,2,9
2,circle,octagon,2,508
3,circle,pentagon,2,1
4,circle,rect,2,18
5,circle,square,2,235
6,circle,triangle,2,2
7,circle,triangle_inverted,2,8
8,diamond,circle,2,18
10,diamond,octagon,2,1
12,diamond,pentagon,2,1


In [82]:
grouped_df = mv_df.groupby(by=['shape', 'predicted_shape', 'group']).count()
grouped_df = grouped_df.reset_index()
grouped_df[grouped_df['group'] == 3][['shape', 'predicted_shape', 'group', 'filename']].rename({'filename':'count'}, axis=1)

,shape,predicted_shape,group,count
11,diamond,other,3,433
19,octagon,other,3,56
23,other,other,3,588
30,pentagon,other,3,751
39,rect,other,3,14723
48,triangle,other,3,149
52,triangle_inverted,other,3,100


In [83]:
# contour detector
mv_df['shape'].value_counts()

rect                 17612
circle                5830
diamond               1726
pentagon              1102
other                  769
triangle_inverted      715
triangle               587
octagon                 63
Name: shape, dtype: int64

In [84]:
# resnet
mv_df['predicted_class'].value_counts().sort_values(ascending=False)

other-0.0-0.0               16800
circle-750.0                 6245
diamond-915.0                1155
square-600.0                  899
triangle_inverted-1220.0      718
octagon-915.0                 574
rect-458.0-610.0              495
triangle-900.0                491
rect-762.0-915.0              472
rect-915.0-1220.0             237
diamond-600.0                 227
pentagon-915.0                 91
Name: predicted_class, dtype: int64